# Practical 3: Projections and Transformations

This is the third practical session and now we're going to get into more of the geography-specific technical details. While this topic may seem less-glamorous than something on analysis, it's possibly the most important one if we want to make sure our analyses are accurate. There are lots of opportunities for pitfalls and errors to creep into our work when we deal with coordinates. We will look at different geographic and projected coordinate reference systems. Then we will see how those are implemented with GeoPandas so that we can reproject GeoDataFrames.

Objectives:
* Know were to find coordinate reference system information
* Identify key projections for working with world and U.K.-based datasets
* Understand the potential distortions caused by certain projections
* View, define, and reproject geospatial vector data


In [ ]:
# load packages
import pyproj # used internally by GeoPandas, but we'll use this for a few initial tests
import geopandas as gpd
import math

## Key definitions

1. **Projection** - a systematic transformation of coordinates. Typically a map projection relates latitudes and longitudes from the globe (3D) to a plane (2D) surface.

2. **Coordinate Reference System (CRS)** - defines how map coordinates relate to actual places on the earth.
3. **Ellipspoid** - 
4. **Datum** - 

Note, these concepts aren't just for earth! There are a large set of coordinate systems for mapping things on mars and other planets, too.

## Coordinate reference systems

Let's begin by looking at the information stored in a CRS. There are many different possible options, and remember that [https://spatialreference.org/](https://spatialreference.org/) is a good reference.

We can load a CRS using `pyproj`.

In [ ]:
# load a global, latitude/longitude CRS
crs1 = pyproj.CRS.from_epsg(4326)

crs1

There are few things to notice in the code above. First, we're using an *EPSG* code. This is the European Petroleum Survey Group's code numbers to reference to geographic coordinate system, and over the years, they've become the standard for referring to spatial reference system. You can find more about there here: [https://epsg.org/home.html](https://epsg.org/home.html)

Second, when we print the CRS, we see several lines with names and what look like parameters. Take a moment to look over these. The EPSG code we are using is *4326*. This is also named *WGS 84*. This is one of the most common CRS and you will encounter it frequently when working with latitude and longitude coordinates. Most GPS units by default map locations using WGS84.

Also note, WGS84 is the name for an Ellipsoid. The names that we see in this CRS are pointing to their own files with parameter values for things like the semi-major and semi-minor axes of a ellipse used to model the shape of the earth.

We see that the *ellipsoid* is listed. It also suggests a bounds. Certain CRS are only appropriate for small areas of interest. As you see, this one is for the entire globe.

If we want to see the details of the parameters, we can convert the CRS object into a well-known text format.

In [ ]:
crs1.to_wkt()

The WKT information is what is stored in the `.prj` file associated with a shapefile.

Let's try inspecting a few more:

In [ ]:
# a popular system for the UK
crs2 = pyproj.CRS.from_epsg(27700)

crs2

Unlike WGS84, we now have a *projected* coordinate system. It is not designed for mapping points on a globe, but for projected coordinates on a flat map. EPSG:27700 is the Ordnance Survey National Grid reference.

This CRS gives us some more detailed information that we didn't see before. There is a *datum* as well as an *ellipsoid* specified. And there is a much smaller suggested area of use.

Note especially the "Axis Info" section. When we are using *projected* coordinates we no longer have latitude and longitude. These now become *easterings* and *northings*. Those measurements specify a distance away from an origin point. These measurements also have defined units: metres.

As we will see in a moment, projected CRS are what we should use for accurate measurements of things like area or distance.

Another one:

In [ ]:
# used by many web-mapping systems (e.g. Google maps)
crs3 = pyproj.CRS.from_epsg(3857)

crs3

EPSG 3857 is used by many of the tiled map services. Despite covernig the whole world, notice that this is, in fact, a projected coordinate system. It has eastings and northings. Google maps, and others like that, aren't globes and they have a lot of distortions around the poles, as we will see.

In later practicals when we want to overlay our geographic data with some basemap imagery or if we want to work with web mapping, we'll need to remember to use this (or similar) projections.

In [ ]:
# a popular global measurement system - place-secific
crs4 = pyproj.CRS.from_epsg(32618)

crs4

The third example is part of the *Universal Transverse Mercator* or UTM projection. We saw an example of this in the lecture materials. UTM has a set of *zones* covering the globe in order to create a local *meridian* to improve the accuracy of local maps. Also notice the line about the Datum. UTM are built referencing the same WGS84 ellipsoid that we saw in the first example. 

There isn't a single "UTM" zone or projection for the entire globe. You need to know where (roughly) you are in order to get your UTM zone and the correct projection.

### Side track: Finding your UTM zone

One of my colleagues recently asked me, "I have a longitude/latitude location, how do I know which UTM zone to use?" 

Yes, you can look up on some of the maps of zones and eyeball which one to use and then look up the EPSG code on the websites. But UTM are actually incredibly regular, so you can identify the zone (and the EPSG) code with a simple calculation.

In [ ]:
# function to return the UTM EPSG
def get_utm_zone(lon, lat):
    utm_band = str((math.floor((lon + 180) / 6) % 60) + 1)
    if len(utm_band) == 1:
        utm_band = '0' + utm_band
    if lat >= 0:
        epsg_code = '326' + utm_band
    else:
        epsg_code = '327' + utm_band
    return epsg_code            

UTM zones are 6 degrees wide at the equator and there are 60 of them covering the globe. Some careful rounding and then differentiating northern vs. southern hemisphere codes. You can try substituting different coordinates in the function, too.

In [ ]:
# test UTM function for the location of the UoS Geography Building
zn = get_utm_zone(-1.399073, 50.934358)
zn

In [ ]:
# print CRS info
pyproj.CRS.from_epsg(zn)

## Which CRS or projection to use?

This is a common question for people starting out working with spatial data. There isn't one correct answer. Most of the time you will receive data and it will already have a CRS and/or projection defined. If you know you are working in a particular country (or state in the U.S.) you may want to re-project to a local system. It will depend on what sort of analyses you want to do.

In much of the rest of this practical we will try to demonstrate the tradeoffs and decisions you will need to make in order to which CRS and projection to use.

## Reprojecting geographic data

Let's imagine that we have some geographic data. A point with latitude and longitude coordinates. We're now going to see how these CRS files can be used to move between a geographic and a projected coordinate reference system.

We will start by doing a transformation manually. Then we'll move on to using the tools in `pyproj`. Then it will be time to return to `GeoPandas` and bring in full `GeoDataFrames`.

### Manual reprojection example

Note: this is just a demonstration. Don't worry if the details of the functions aren't clear, and you won't need them to complete the practicals. To see more of this process explained, check out this reference: [http://fgg-web.fgg.uni-lj.si/~/mkuhar/Zalozba/TM_projection.pdf](http://fgg-web.fgg.uni-lj.si/~/mkuhar/Zalozba/TM_projection.pdf)

First we'll define a few functions.

In [ ]:
# based on: https://scipython.com/book/chapter-2-the-core-python-language-i/additional-problems/converting-between-an-os-grid-reference-and-longitudelatitude/
# initial parameter steps
def fM(phi, a, b):
    """Return the parameter M for latitude phi using ellipsoid params a, b."""

    n = (a-b)/(a+b)
    n2 = n**2
    n3 = n * n2
    dphi, sphi = phi - phi0, phi + phi0
    M = b * F0 * (
            (1 + n + 5/4 * (n2+n3)) * dphi
          - (3*n + 3*n2 + 21/8 * n3) * math.sin(dphi) * math.cos(sphi)
          + (15/8 * (n2 + n3)) * math.sin(2*dphi) * math.cos(2*sphi)
          - (35/24 * n3 * math.sin(3*dphi) * math.cos(3*sphi))
        )
    return M

# key parameters
def get_prms(phi, a, F0, e2):
    """Calculate and return the parameters rho, nu, and eta2."""

    rho = a * F0 * (1-e2) * (1-e2*math.sin(phi)**2)**-1.5
    nu = a * F0 / math.sqrt(1-e2*math.sin(phi)**2)
    eta2 = nu/rho - 1
    return rho, nu, eta2

# main conversion function
def ll_to_os(phi, lam, datum='osgb36'):
    """Convert from latitude and longitude to OS grid reference (E, N).

    Latitude, phi, and longitude, lambda, are to be provided in degrees.

    """

    phi, lam = math.radians(phi), math.radians(lam)
    a, b = datum_ellipsoid[datum]['a'], datum_ellipsoid[datum]['b']
    e2 = (a**2 - b**2)/a**2
    rho, nu, eta2 = get_prms(phi, a, F0, e2)
    M = fM(phi, a, b)

    sin_phi = math.sin(phi)
    cos_phi = math.cos(phi)
    cos_phi2 = cos_phi**2
    cos_phi3 = cos_phi2 * cos_phi
    cos_phi5 = cos_phi3 * cos_phi2
    tan_phi2 = math.tan(phi)**2
    tan_phi4 = tan_phi2 * tan_phi2

    a1 = M + N0
    a2 = nu/2 * sin_phi * cos_phi
    a3 = nu/24 * sin_phi * cos_phi3 * (5 - tan_phi2 + 9*eta2)
    a4 = nu/720 * sin_phi * cos_phi5 * (61 - 58*tan_phi2 + tan_phi4)
    b1 = nu * cos_phi
    b2 = nu/6 * cos_phi3 * (nu/rho - tan_phi2)
    b3 = nu/120 * cos_phi5 * (5 - 18*tan_phi2 + tan_phi4 + eta2*(14 -
                              58*tan_phi2))
    lml0 = lam - lambda0
    lml02 = lml0**2
    N = a1 + lml02 * (a2 + lml02*(a3 + a4*lml02))
    E = E0 + lml0 * (b1 + lml02*(b2 + b3*lml02))
    return E, N

Next we need to define a few constants describing the different parameters of the Ellipsoids. This is the kind of information stored in those CRS files that are referenced by `pyproj`.

In [ ]:
# Ellipsoid parameters for different datums (m): semi-major axis, a, and
# semi-minor axis, b.
datum_ellipsoid = {
    # Airy 1830 ellipsoid
    'osgb36': {'a': 6.377563396e6,
               'b': 6.356256909e6
              },
    # WGS84 ellipsoid parameters
    'wgs84':  {'a': 6.378137e6,
               'b': 6.3567523141e6
              },
    }

# Transverse Mercator projection parameters: Map coordinates of true origin,
# (E0, N0), scale factor on central meridian, F0, true origin (phi0, lambda0).
N0 = -100000
E0 = 400000
F0 = 0.9996012717
phi0 = math.radians(49)
lambda0 = math.radians(-2)

But these equations are **MISSING** something. To make them simpler they assume that the two CRS are using the same *datum*. By not including a transformation it means there will be slight inaccuracies when we move from something like WGS84 to the OS National Grid. EPSG 27700 uses OSGB 1936 ellipsoid with Airy 1830 datum. 

Let's use the location of the UoS Geography building again.

In [ ]:
# convert lat, lon (phi, lambda) to OS Easting, Northings
# note the order of the coordinates
ll_to_os(50.934358, -1.399073)

# remember, these units are now in metres

### Reprojecting example with `pyproj`

The previous example of calculating the transformation is complicated and there are lots of opportunities for us to make mistakes. There are also lots of small details to remember like datums and ellipsoids and transformations.

Luckily, we rarely will ever need to do this kind of reprojection step manually and we have many tools to make this sort of thing easier. Let's now repeat this transformation using `pyproj`.

First, create a `pyproj.Transformer` that controls the reprojection between two defined CRS. Here we will EPSG codes that we've already seen before.

In [ ]:
from pyproj import Transformer
# create a transformer object
# using the EPSG codes again
transformer = Transformer.from_crs(4326, 27700, always_xy=True)
# the `always_xy` argument enforces lon, lat order 

# alternatively, we could have use the saved CRS objects from above, (e.g. crs3) instead of EPSG codes

In [ ]:
# it's simple to use the transformer with a coordinate pair!
# note the order of lon, lat
transformer.transform(-1.399073, 50.934358)

Compare this result with the manual calculation we just did. Very close! As noted above, the difference is coming mainly because `pyproj` knows to apply the correct transformation switching the datums.

## Coordinate reference systems in `GeoPandas`

The tools with `GeoPandas` makes working with coordinate reference systems relatively easy. When data are read into a `GeoDataFrame` the CRS information should automatically be added from the file.

In [ ]:
# read in a sample dataset
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [ ]:
# retrieve the CRS information
world.crs

In this example, we can access the CRS through the `.crs` attribute of a GeoDataFrame. These data happen to be in EPSG:4326, or WGS 1984, which we saw in the section above.

What if we tried to modify that attribute?

In [ ]:
# create a (deep) copy
world_cpy = world.copy()
# set the `.crs` to something else
world_cpy.crs = pyproj.CRS.from_epsg(3857)

world_cpy.crs

Python let's us do this, but **THIS IS INCORRECT**. The data have not been transformed, but GeoPandas now thinks the degrees of latitude and longitude are measurements taken in metres. We've shrunk the entire world!

Now let's try doing this the correct way by reprojecting these data.

In [ ]:
# create a new GeoDataFrame to store the results so we can compare
# project to Web Mercator (EPSG:3857)
world_wm = world.to_crs(epsg=3857)  # can also use: .to_crs("EPSG:3857")

In [ ]:
# compare the results
world_wm.crs

In [ ]:
# Let's compare some of the coordinates
print(world.geometry.head())

In [ ]:
# and after projections
print(world_wm.geometry.head())

Notice the numbers are completely different now. The decimal degrees of latitude/longitude have been transformed into metres. This is the basic fact why the CRS is so important - if you want to overlay two datasets or to interact with them, you have to consider how they map their location on earth. Otherwise there will be problems in your analyses.

The difference in these maps is even more apparent when we visualise them side-by-side. This also gives us an opportunity to test out how to manipulate multiple plots, which we will do more of later.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# start sub-plots for side-by-side maps
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12, 8))

# plot the original data
world.plot(ax=ax1)
ax1.set_title("WGS 84")

# plot the reprojected data
world_wm.plot(ax=ax2)
ax2.set_title("Web Mercator")

# remove whitespace
plt.tight_layout

### Testing different `CRS`

Based on the model above, try reprojecting and then plotting the `world` dataset into a few of the other CRS using `.to_crs`. Store the reprojected datasets in new GeoDataFrames so we can compare them in the next section. You can use the EPSG codes that we tested above, or look for some others.

In [ ]:
# Try some different options here

#

In [ ]:
# Try some different options here

#

Option 2: what happens when you use a local UTM projection on a global dataset? (absolutely huge distortion, globally. Great locally)

In [ ]:
# use the UTM zone we calculated above for Southampton
world_utm = world.to_crs(32630)

world_utm.plot()

## Distortions in projections

We've already seen that the choice of projection can impact the appearance of the map. Certain projections distort parts of the globe, stretching them much larger than they are actually.

### Measuring areas

The distortions also show up when we're trying to calculate certain attributes of the spatial datasets. Here we will look at the area measurements, and in a later lesson we'll look at distance in more detail.

Remember that some projections are optimised to preserve certain properties, like area.

In [ ]:
# check the initial area estimates for the UK
world.loc[world.name=='United Kingdom'].area

In [ ]:
# compare this with one of the projected coordinate systems
world_wm.loc[world_wm.name=='United Kingdom'].area  # units in sq. m

These area results will now be in the units of the CRS. Of course, if we want a really accurate area measurement then we also need to consider the scale of our data. These country shapes have been generalised - the edges are simplified - so they aren't the best data to use to make a really accurate area measurement. 

In [ ]:
# compare the area calculations with some of your projected datasets.

#

### Global coordinate systems

Thus far we have been most concerned about plotting 2D maps and the distortions in them. Is it possible to plot data so that it looks more 3D? 

For that we need something called an "orthographic" projection.

In [ ]:
# create a custom proj string
# based on https://automating-gis-processes.github.io/site/notebooks/LS/projections.html
ortho = pyproj.CRS.from_proj4("+proj=ortho +lat_0=50.00 +lon_0=-1.13 +x_0=0 +y_0=0 +a=6370997 +b=6370997 +units=m +no_defs")
# I have set the central point to be approximately over Southampton (50, -1.1)

In [ ]:
# reproject the world and plot
world_ortho = world.to_crs(ortho)

world_ortho.plot()

plt.axis('off')
plt.title("Orthographic")

### Alternative projections using `cartopy`

We can have access to even more projections (and some unique projections) by using another package called `cartopy`. Inside it uses a slightly different set-up from `pyproj`, so we will have to do some conversion steps but it still works with GeoPandas.

See more here: [https://geopandas.readthedocs.io/en/latest/gallery/cartopy_convert.html](https://geopandas.readthedocs.io/en/latest/gallery/cartopy_convert.html)

In [ ]:
# load package
from cartopy import crs as ccrs

In [ ]:
# create a CRS from `cartopy`
crs_new = ccrs.InterruptedGoodeHomolosine()

# loop through each geometry and calculate the projection
new_geometries = [crs_new.project_geometry(ii, src_crs=ccrs.Mercator())
                  for ii in world_wm['geometry'].values]

In [ ]:
# we now have a list of new `shapely` geometries
# cartopy allows us to plot the results
fig, ax = plt.subplots(subplot_kw={'projection': crs_new})
ax.add_geometries(new_geometries, crs=crs_new)

In [ ]:
# these geometry data can be used to create a new GeoDataFrame
# start with the web mercator projection, not the WGS84 geographic coord system dataset
world_hs = gpd.GeoDataFrame(world_wm.drop('geometry', axis=1),  # just use the attributes
                            geometry = new_geometries,  # add in the new projected geometry
                            crs = crs_new.proj4_init    # convert cartopy's CRS to something GPD understands
                           )

world_hs.head()

In [ ]:
# plot the results
world_hs.plot(facecolor='sandybrown', edgecolor='black')
# not quite as pretty as with cartopy

### Optional (if you have time): Try another `cartopy` projection

Follow the model from above, but try a different projection from the available ones here: [https://scitools.org.uk/cartopy/docs/latest/crs/projections.html](https://scitools.org.uk/cartopy/docs/latest/crs/projections.html)

In [ ]:
# Testing space


#

Finished!